In [1]:
import pandas as pd
import numpy as np
from sklearn import linear_model
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_samples, silhouette_score
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectFromModel
from sklearn.model_selection import ShuffleSplit
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import seaborn as sns
import matplotlib.style as style
style.use('fivethirtyeight')

In [2]:
orders = pd.read_csv('Orders.csv',index_col='Row.ID',
                     parse_dates=['Order.Date','Ship.Date'])

returns = pd. read_csv("Returns.csv")

In [3]:
print(orders.shape)
print(returns.shape)

(51290, 23)
(1079, 3)


In [4]:
orders.columns.to_list()

['Order.ID',
 'Order.Date',
 'Ship.Date',
 'Ship.Mode',
 'Customer.ID',
 'Customer.Name',
 'Segment',
 'Postal.Code',
 'City',
 'State',
 'Country',
 'Region',
 'Market',
 'Product.ID',
 'Category',
 'Sub.Category',
 'Product.Name',
 'Sales',
 'Quantity',
 'Discount',
 'Profit',
 'Shipping.Cost',
 'Order.Priority']

In [5]:
orders = orders.astype({'Sales':'string', 'Profit':'string'})

In [6]:
orders['Sales'] = orders.apply(lambda x: x['Sales'].replace('$',''), axis = 1)

In [7]:
orders['Profit'] = orders.apply(lambda x: x['Profit'].replace('$',''), axis = 1)

In [8]:
orders['Sales'] = orders.apply(lambda x: x['Sales'].replace(',',''), axis = 1)
orders['Profit'] = orders.apply(lambda x: x['Profit'].replace(',',''), axis = 1)

In [9]:
orders.astype({'Sales':'float64', 'Profit':'float64'})

,Order.ID,Order.Date,Ship.Date,Ship.Mode,Customer.ID,Customer.Name,Segment,Postal.Code,City,State,...,Product.ID,Category,Sub.Category,Product.Name,Sales,Quantity,Discount,Profit,Shipping.Cost,Order.Priority
Row.ID,,,,,,,,,,,,,,,,,,,,,
40098,CA-2014-AB10015140-41954,2014-11-11,2014-11-13,First Class,AB-100151402,Aaron Bergman,Consumer,73120.0,Oklahoma City,Oklahoma,...,TEC-PH-5816,Technology,Phones,Samsung Convoy 3,221.98,2,0.0,62.15,40.770,High
26341,IN-2014-JR162107-41675,2014-02-05,2014-02-07,Second Class,JR-162107,Justin Ritter,Corporate,NaN,Wollongong,New South Wales,...,FUR-CH-5379,Furniture,Chairs,"Novimex Executive Leather Armchair, Black",3709.40,9,0.1,-288.77,923.630,Critical
25330,IN-2014-CR127307-41929,2014-10-17,2014-10-18,First Class,CR-127307,Craig Reiter,Consumer,NaN,Brisbane,Queensland,...,TEC-PH-5356,Technology,Phones,"Nokia Smart Phone, with Caller ID",5175.17,9,0.1,919.97,915.490,Medium
13524,ES-2014-KM1637548-41667,2014-01-28,2014-01-30,First Class,KM-1637548,Katherine Murray,Home Office,NaN,Berlin,Berlin,...,TEC-PH-5267,Technology,Phones,"Motorola Smart Phone, Cordless",2892.51,5,0.1,-96.54,910.160,Medium
47221,SG-2014-RH9495111-41948,2014-11-05,2014-11-06,Same Day,RH-9495111,Rick Hansen,Consumer,NaN,Dakar,Dakar,...,TEC-CO-6011,Technology,Copiers,"Sharp Wireless Fax, High-Speed",2832.96,8,0.0,311.52,903.040,Critical
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29002,IN-2015-KE1642066-42174,2015-06-19,2015-06-19,Same Day,KE-1642066,Katrina Edelman,Corporate,NaN,Kure,Hiroshima,...,OFF-FA-3072,Office Supplies,Fasteners,"Advantus Thumb Tacks, 12 Pack",65.10,5,0.0,4.50,1.010,Medium
34337,US-2014-ZD21925140-41765,2014-05-06,2014-05-10,Standard Class,ZD-219251408,Zuschuss Donatelli,Consumer,37421.0,Chattanooga,Tennessee,...,FUR-FU-4070,Furniture,Furnishings,"Eldon Image Series Desk Accessories, Burgundy",16.72,5,0.2,3.34,1.930,High
31315,CA-2012-ZD21925140-41147,2012-08-26,2012-08-31,Second Class,ZD-219251404,Zuschuss Donatelli,Consumer,94109.0,San Francisco,California,...,OFF-AR-5321,Office Supplies,Art,Newell 341,8.56,2,0.0,2.48,1.580,High


In [10]:
orders.loc[:,['Order.ID', 'Order.Date', 'Product.ID',
 'Category',
 'Sub.Category',
 'Product.Name',
 'Sales',
 'Quantity',
 'Discount',
 'Profit',]].head(10)

,Order.ID,Order.Date,Product.ID,Category,Sub.Category,Product.Name,Sales,Quantity,Discount,Profit
Row.ID,,,,,,,,,,
40098,CA-2014-AB10015140-41954,2014-11-11,TEC-PH-5816,Technology,Phones,Samsung Convoy 3,221.98,2,0.0,62.15
26341,IN-2014-JR162107-41675,2014-02-05,FUR-CH-5379,Furniture,Chairs,"Novimex Executive Leather Armchair, Black",3709.40,9,0.1,-288.77
25330,IN-2014-CR127307-41929,2014-10-17,TEC-PH-5356,Technology,Phones,"Nokia Smart Phone, with Caller ID",5175.17,9,0.1,919.97
13524,ES-2014-KM1637548-41667,2014-01-28,TEC-PH-5267,Technology,Phones,"Motorola Smart Phone, Cordless",2892.51,5,0.1,-96.54
47221,SG-2014-RH9495111-41948,2014-11-05,TEC-CO-6011,Technology,Copiers,"Sharp Wireless Fax, High-Speed",2832.96,8,0.0,311.52
22732,IN-2014-JM156557-41818,2014-06-28,TEC-PH-5842,Technology,Phones,"Samsung Smart Phone, with Caller ID",2862.68,5,0.1,763.28
30570,IN-2012-TS2134092-41219,2012-11-06,FUR-CH-5378,Furniture,Chairs,"Novimex Executive Leather Armchair, Adjustable",1822.08,4,0.0,564.84
31192,IN-2013-MB1808592-41378,2013-04-14,FUR-TA-3764,Furniture,Tables,"Chromcraft Conference Table, Fully Assembled",5244.84,6,0.0,996.48
40099,CA-2014-AB10015140-41954,2014-11-11,FUR-BO-5957,Furniture,Bookcases,"Sauder Facets Collection Library, Sky Alder Fi...",341.96,2,0.0,54.71


In [11]:
orders['Order.Yr'] = orders.apply(lambda x: x['Order.Date'].year, axis=1)
orders['Order.Mnth'] = orders.apply(lambda x: x['Order.Date'].month, axis=1)

In [12]:
orders.dtypes

Order.ID                  object
Order.Date        datetime64[ns]
Ship.Date         datetime64[ns]
Ship.Mode                 object
Customer.ID               object
Customer.Name             object
Segment                   object
Postal.Code              float64
City                      object
State                     object
Country                   object
Region                    object
Market                    object
Product.ID                object
Category                  object
Sub.Category              object
Product.Name              object
Sales                     object
Quantity                   int64
Discount                 float64
Profit                    object
Shipping.Cost            float64
Order.Priority            object
Order.Yr                   int64
Order.Mnth                 int64
dtype: object

In [13]:
# months = [f'{m[0]}-{m[1]}'for m in list(orders.groupby(['Order.Yr', 'Order.Mnth']).sum().loc[:,['Sales', 'Quantity']].index)]

In [14]:
# x = months
# y = orders.groupby(['Order.Yr', 'Order.Mnth']).sum().loc[:,['Sales', 'Quantity']]['Sales']
# _ = sns.lineplot(x=x,y=y)
# _.tick_params(axis='x', rotation=90, labelsize=7)
# plt.show()

In [15]:
# x = months
# y = orders.groupby(['Order.Yr', 'Order.Mnth']).sum().loc[:,['Sales', 'Quantity']]['Quantity']
# _ = sns.lineplot(x=x,y=y)
# _.tick_params(axis='x', rotation=90, labelsize=7)
# plt.show()

In [16]:
# bycat_data = orders.groupby(['Order.Yr', 'Order.Mnth', 'Category']).sum().loc[:,['Sales', 'Quantity']].reset_index()
# sns.lineplot(data=bycat_data, x = 'Order.Mnth', y='Sales', hue='Category')

In [17]:
# bycat_data = orders.groupby(['Order.Yr', 'Order.Mnth', 'Category']).sum().loc[:,['Sales', 'Quantity']].reset_index()
# sns.lineplot(data=bycat_data, x = 'Order.Mnth', y='Quantity', hue='Category')

In [18]:
returns.columns.to_list()

['Returned', 'Order ID', 'Region']

In [19]:
OR_merge = orders.merge(returns, how='left', left_on='Order.ID', right_on='Order ID')
OR_merge.head()

,Order.ID,Order.Date,Ship.Date,Ship.Mode,Customer.ID,Customer.Name,Segment,Postal.Code,City,State,...,Quantity,Discount,Profit,Shipping.Cost,Order.Priority,Order.Yr,Order.Mnth,Returned,Order ID,Region_y
0,CA-2014-AB10015140-41954,2014-11-11,2014-11-13,First Class,AB-100151402,Aaron Bergman,Consumer,73120.0,Oklahoma City,Oklahoma,...,2,0.0,62.15,40.77,High,2014,11,NaN,NaN,NaN
1,IN-2014-JR162107-41675,2014-02-05,2014-02-07,Second Class,JR-162107,Justin Ritter,Corporate,NaN,Wollongong,New South Wales,...,9,0.1,-288.77,923.63,Critical,2014,2,NaN,NaN,NaN
2,IN-2014-CR127307-41929,2014-10-17,2014-10-18,First Class,CR-127307,Craig Reiter,Consumer,NaN,Brisbane,Queensland,...,9,0.1,919.97,915.49,Medium,2014,10,NaN,NaN,NaN
3,ES-2014-KM1637548-41667,2014-01-28,2014-01-30,First Class,KM-1637548,Katherine Murray,Home Office,NaN,Berlin,Berlin,...,5,0.1,-96.54,910.16,Medium,2014,1,NaN,NaN,NaN
4,SG-2014-RH9495111-41948,2014-11-05,2014-11-06,Same Day,RH-9495111,Rick Hansen,Consumer,NaN,Dakar,Dakar,...,8,0.0,311.52,903.04,Critical,2014,11,NaN,NaN,NaN


In [20]:
OR_merge.shape

(51290, 28)

In [21]:
OR_merge.isna().sum()

Order.ID              0
Order.Date            0
Ship.Date             0
Ship.Mode             0
Customer.ID           0
Customer.Name         0
Segment               0
Postal.Code       41296
City                  0
State                 0
Country               0
Region_x              0
Market                0
Product.ID            0
Category              0
Sub.Category          0
Product.Name          0
Sales                 0
Quantity              0
Discount              0
Profit                0
Shipping.Cost         0
Order.Priority        0
Order.Yr              0
Order.Mnth            0
Returned          49070
Order ID          49070
Region_y          49070
dtype: int64

In [22]:
OR_merge.loc[OR_merge['Returned'].isna(),'return_flag'] = 0
OR_merge.loc[~OR_merge['Returned'].isna(),'return_flag'] = 1

In [25]:
OR_merge.dtypes

Order.ID                  object
Order.Date        datetime64[ns]
Ship.Date         datetime64[ns]
Ship.Mode                 object
Customer.ID               object
Customer.Name             object
Segment                   object
Postal.Code              float64
City                      object
State                     object
Country                   object
Region_x                  object
Market                    object
Product.ID                object
Category                  object
Sub.Category              object
Product.Name              object
Sales                     object
Quantity                   int64
Discount                 float64
Profit                    object
Shipping.Cost            float64
Order.Priority            object
Order.Yr                   int64
Order.Mnth                 int64
Returned                  object
Order ID                  object
Region_y                  object
return_flag              float64
dtype: object

In [23]:
OR_merge.groupby('return_flag').sum()

,Postal.Code,Quantity,Discount,Shipping.Cost,Order.Yr,Order.Mnth
return_flag,,,,,,
0.0,525128350.0,170637,7003.200,1.297243e+06,98816106,368448
1.0,26444302.0,7675,326.528,6.084260e+04,4470527,17049


In [24]:
OR_merge.groupby('return_flag').sum()['Profit']

KeyError: 'Profit'

In [ ]:
ret_cnt = OR_merge.groupby('Customer.ID').sum()['return_flag']

In [ ]:
sum(ret_cnt > 1)

In [ ]:
sum(ret_cnt > 5)

In [ ]:
OR_merge.dtypes

In [ ]:
OR_merge['process_time'] = OR_merge['Ship.Date'] - OR_merge['Order.Date']
OR_merge.groupby(['return_flag']).agg({'process_time':'mean'})